In [3]:
import pandas as pd
import pickle

import sys
import os

# sys.path.append(os.path.abspath(os.path.join('../src')))  # Adjust as needed

from src.distance_utils import distance
from src.data_load import load_tables
from src.filtering import filter_labors_by_date, filter_labors_by_city, filter_labores
from src.metrics import collect_vt_metrics_range, show_day_report_dayonly, compute_indicators
from src.preprocessing import remap_to_base_date, build_services_map_df, process_group
from src.plotting import plot_results
from src.config import *
from src.algorithms import remove_drivers, compute_avg_times, run_iteration_chronological, init_drivers

from src.config import *

data_path = '../data'

directorio_df, df_dist, cities_df, duraciones_df, valid_cities = load_tables(data_path)

In [5]:
# =========================
# 2) EJECUCIÓN COMPLETA
# =========================
# (a) Rango y métricas del rango
start_date = "2025-07-01"
end_date   = "2025-07-28"

metrics_df, fig_metrics, fig_counts = collect_vt_metrics_range(
    df=df_dist,           # tu dataframe completo
    start_date=start_date,
    end_date=end_date,
    workday_hours=8,
    city_code=149,        # Bogotá
    skip_weekends=True
)

display(metrics_df)
if fig_metrics: fig_metrics.show()  # Labores/Conductor + Utilización promedio
if fig_counts:  fig_counts.show()   # N° Conductores + N° Labores



#    Opción 2: por fecha exacta dentro del rango
day_report = show_day_report_dayonly(
    df_full=df_dist,
    metrics_df=metrics_df,
    day_str="2025-07-15",
    workday_hours=8,
    city_code=149,
    only_vt=True
)

,day,vt_count,num_drivers,labores_por_conductor,utilizacion_promedio_%
0,2025-07-01,104,33,3.152,58.3
1,2025-07-02,133,36,3.694,71.1
2,2025-07-03,111,34,3.265,64.3
3,2025-07-04,127,33,3.848,69.1
4,2025-07-07,127,35,3.629,70.2
5,2025-07-08,131,36,3.639,75.5
6,2025-07-09,120,34,3.529,68.2
7,2025-07-10,129,31,4.161,75.0
8,2025-07-11,118,37,3.189,67.0
9,2025-07-14,131,35,3.743,64.6


Reporte del día: 2025-07-15


,Métrica,Valor
0,Labores VEHICLE_TRANSPORTATION (día filtrado),112
1,Conductores únicos (alfred),33
2,Labores por conductor,3.394
3,Utilización promedio conductores,65.8%


,alfred,total_min,total_horas,utilizacion_%,tiempo_extra_min,tiempo_extra_horas
0,3214.0,562.8,9.38,117.2,82.8,1.38
1,20111.0,514.0,8.57,107.1,34.0,0.57
2,72680.0,511.0,8.52,106.5,31.0,0.52
3,11714.0,508.9,8.48,106.0,28.9,0.48
4,5455.0,471.0,7.85,98.1,0.0,0.00
5,4554.0,445.0,7.42,92.7,0.0,0.00
6,6229.0,440.6,7.34,91.8,0.0,0.00
7,211.0,423.0,7.05,88.1,0.0,0.00
8,26834.0,413.0,6.88,86.0,0.0,0.00
9,10451.0,384.9,6.42,80.2,0.0,0.00


In [3]:
# Load datasets
# df_dist = pd.read_pickle("df_dist.pkl")

df_bogota = filter_labors_by_city(df_dist, "149")
df_dia = filter_labors_by_date(df_bogota, start_date="2023-01-02", end_date="2023-01-03")
df_dia = df_dia.query("state_service!='CANCELED'").sort_values(['service_id','labor_start_date']).reset_index(drop=True)

# Remap dates
base_day = pd.to_datetime("2023-01-02").date()
df_dia = remap_to_base_date(df_dia, ['schedule_date', 'labor_start_date', 'labor_end_date'], base_day)

# Build mapping
services_map_df = build_services_map_df(df_dia)

# Process groups
with open('distances.pkl', 'rb') as f:
    DIST_DICT = pickle.load(f)

cleaned = [process_group(grp, DISTANCE_METHOD, DIST_DICT) for _, grp in df_dia.groupby('service_id', sort=False)]
df_cleaned_template = pd.concat([c for c in cleaned if not c.empty], ignore_index=True)
df_cleaned_template = df_cleaned_template.merge(services_map_df, on=['service_id','labor_id'], how='left')

# Save
df_cleaned_template.to_pickle('cleaned_static_template.pkl')

# Algorithmo estático

In [4]:
# — Ejecución del algoritmo estático ————————————————————————————————————————————————
# nombres_a_quitar = ["Manuel Romero", "Jorge Cortes Rojas", "Orlando Jacinto"]
# directorio_df = remove_drivers(directorio_df, nombres_a_quitar)

# — Día de prueba ————————————————————————————————————————————————
day_str = "2024-10-21"  # día con baja carga

template_path       = f"{data_path}/data_clean/templates/cleaned_static_template_{day_str}.pkl"
df_cleaned_template = pd.read_pickle(template_path)

# with open(f'{data_path}/data_clean/distances.pkl','rb') as f:
#     DIST_DICT = pickle.load(f)

df_cleaned_template = filter_labores(df_cleaned_template)
avg_times_map = compute_avg_times(df_dist)

# — Ejecutar —————————————————————————————————————————————————————————
df_result, df_moves, n_drivers = run_iteration_chronological(  alpha=ALPHA, df_cleaned_template=df_cleaned_template,
                                                    tiempo_previo=TIEMPO_PREVIO, 
                                                    tiempo_gracia=TIEMPO_GRACIA, tiempo_alistar=TIEMPO_ALISTAR,
                                                    tiempo_finalizacion=TIEMPO_FINALIZACION, 
                                                    vehicle_transport_speed=VEHICLE_TRANSPORT_SPEED, 
                                                    alfred_speed=ALFRED_SPEED, 
                                                    avg_times_map=avg_times_map, tiempo_other=TIEMPO_OTHER,
                                                    DISTANCE_METHOD=DISTANCE_METHOD, directorio_df=directorio_df,
                                                    day_str=day_str, ciudad='BOGOTA'
                                                  ) 

indicators, ind_df = compute_indicators(df_result, df_moves, n_drivers, TIEMPO_GRACIA)
print(ind_df)
plot_results(df_result, df_moves, day_str, TIEMPO_GRACIA)

                     Indicador  Valor
0                   N° Labores     75
1   N° Conductores Disponibles     20
2               % Tiempo Libre  57.1%
3                   % Fallidas   0.0%
4            N° Llegadas Tarde      4
5           % Tardanza (norm.)  28.4%
6  Prom. Servicios x Conductor   3.10
7               Obj. Ponderado   1.55
